In [1]:
import numpy as np
import pypulseq as pp
import MRzeroCore as mr0
import torch
import matplotlib.pyplot as plt

# HACK: newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float

In [2]:
seq = pp.Sequence()
seq.read("out/flash pTx.seq")

print('load phantom')
# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
sz = [64, 64]
obj_p = mr0.VoxelGridPhantom.load_mat('../data/numerical_brain_cropped.mat')
brain_phantom_res = 64 #@param {type:"slider", min:16, max:128, step:16}
obj_p = obj_p.interpolate(brain_phantom_res, brain_phantom_res, 1)
obj_p.B0[:] = 0
plot_phantom = False #@param {type:"boolean"}
if plot_phantom: obj_p.plot()

obj_p = obj_p.build()
print('simulate (2D) \n' + seq_file)
print(os.getcwd())
seq0_file = mr0.PulseqFile(seq_file)
seq0 = mr0.Sequence.from_seq_file(seq0_file)
# seq0.plot_kspace_trajectory()
# Simulate the sequence
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p)
# PLOT sequence with SIGNAL

sp_adc, t_adc = util.pulseq_plot(seq=seq,signal=signal.numpy())



RuntimeError: Unsupported version_minor: 4. Expected: 3

In [ ]:
#@title 3. reconstruct the image
# Unfortunately, we need to limit the resolution as reco_adjoint is very RAM-hungy
print('reconstruct and plot')
seq0.plot_kspace_trajectory()

reco = mr0.reco_adjoint(signal, seq0.get_kspace(), resolution=(64, 64, 1), FOV=(1, 1, 1))
plt.figure()
plt.subplot(121)
plt.title("Magnitude")
plt.imshow(reco[:, :, 0].T.abs(), origin="lower")
plt.colorbar()
plt.subplot(122)
plt.title("Phase")
plt.imshow(reco[:, :, 0].T.angle(), origin="lower", vmin=-np.pi, vmax=np.pi)
plt.colorbar()
plt.show()